# Pre-Processing of POINT-5 data

## Loading Packages

In [1]:
import os.path
import subprocess

## Download reference fasta and GTF file with annotated transcripts 

In [2]:
!wget http://ftp.ensembl.org/pub/release-90/fasta/homo_sapiens/dna/Homo_sapiens.GRCh38.dna.primary_assembly.fa.gz 
!wget http://ftp.ensembl.org/pub/release-90/gtf/homo_sapiens/Homo_sapiens.GRCh38.90.gtf.gz 
!wget http://hgdownload.soe.ucsc.edu/admin/exe/linux.x86_64/bedGraphToBigWig

--2022-05-23 23:51:02--  http://ftp.ensembl.org/pub/release-90/fasta/homo_sapiens/dna/Homo_sapiens.GRCh38.dna.primary_assembly.fa.gz
Resolving ftp.ensembl.org (ftp.ensembl.org)... 193.62.193.139
Connecting to ftp.ensembl.org (ftp.ensembl.org)|193.62.193.139|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 881214396 (840M) [application/x-gzip]
Saving to: ‘Homo_sapiens.GRCh38.dna.primary_assembly.fa.gz’

Homo_sapiens.GRCh38 100%[===================>] 840.39M  2.36MB/s    in 5m 41s  

2022-05-23 23:56:45 (2.46 MB/s) - ‘Homo_sapiens.GRCh38.dna.primary_assembly.fa.gz’ saved [881214396/881214396]

--2022-05-23 23:56:45--  http://ftp.ensembl.org/pub/release-90/gtf/homo_sapiens/Homo_sapiens.GRCh38.90.gtf.gz
Resolving ftp.ensembl.org (ftp.ensembl.org)... 193.62.193.139
Connecting to ftp.ensembl.org (ftp.ensembl.org)|193.62.193.139|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 41837030 (40M) [application/x-gzip]
Saving to: ‘Homo_sapiens.GRCh38.9

In [3]:
!gunzip Homo_sapiens.GRCh38.dna.primary_assembly.fa.gz 
!gunzip Homo_sapiens.GRCh38.90.gtf.gz

## Download POINT-5 example data

To demonstrate step-by-step the pre-processing of POINT-5 reads, a publicly available POINT-5 sample from [Sousa-Luís et al. Mol Cel. 2021](https://doi.org/10.1016/j.molcel.2021.02.034) will be used ([SRR12802837](https://www.ncbi.nlm.nih.gov/sra/SRX9271661)). This sample profiles POINT-5 on untreated HeLa cells.

In [4]:
!fasterq-dump SRR12802837 --threads 20 --split-files

spots read      : 94,930,617
reads read      : 189,861,234
reads written   : 189,861,234


In [5]:
!pigz -p 20 SRR12802837_1.fastq
!pigz -p 20 SRR12802837_2.fastq

## Quality control - FastQC

In [6]:
!fastqc -t 20 SRR12802837_1.fastq.gz
!fastqc -t 20 SRR12802837_2.fastq.gz

Started analysis of SRR12802837_1.fastq.gz
Approx 5% complete for SRR12802837_1.fastq.gz
Approx 10% complete for SRR12802837_1.fastq.gz
Approx 15% complete for SRR12802837_1.fastq.gz
Approx 20% complete for SRR12802837_1.fastq.gz
Approx 25% complete for SRR12802837_1.fastq.gz
Approx 30% complete for SRR12802837_1.fastq.gz
Approx 35% complete for SRR12802837_1.fastq.gz
Approx 40% complete for SRR12802837_1.fastq.gz
Approx 45% complete for SRR12802837_1.fastq.gz
Approx 50% complete for SRR12802837_1.fastq.gz
Approx 55% complete for SRR12802837_1.fastq.gz
Approx 60% complete for SRR12802837_1.fastq.gz
Approx 65% complete for SRR12802837_1.fastq.gz
Approx 70% complete for SRR12802837_1.fastq.gz
Approx 75% complete for SRR12802837_1.fastq.gz
Approx 80% complete for SRR12802837_1.fastq.gz
Approx 85% complete for SRR12802837_1.fastq.gz
Approx 90% complete for SRR12802837_1.fastq.gz
Approx 95% complete for SRR12802837_1.fastq.gz
Analysis complete for SRR12802837_1.fastq.gz
Started analysis of 

## Trimming reads' adaptors - TrimGalore

In [7]:
!trim_galore --phred33 -q 30 -e 0.05 --illumina --fastqc --paired --length 10 SRR12802837_1.fastq.gz SRR12802837_2.fastq.gz

Multicore support not enabled. Proceeding with single-core trimming.
Path to Cutadapt set as: 'cutadapt' (default)
Cutadapt seems to be working fine (tested command 'cutadapt --version')
Cutadapt version: 4.0
single-core operation.

gzip: stdout: Broken pipe
Writing report to 'SRR12802837_1.fastq.gz_trimming_report.txt'

SUMMARISING RUN PARAMETERS
Input filename: SRR12802837_1.fastq.gz
Trimming mode: paired-end
Trim Galore version: 0.6.7
Cutadapt version: 4.0
Number of cores used for trimming: 1
Quality Phred score cutoff: 30
Quality encoding type selected: ASCII+33
Adapter sequence: 'AGATCGGAAGAGC' (Illumina TruSeq, Sanger iPCR; user defined)
Maximum trimming error rate: 0.05
Minimum required adapter overlap (stringency): 1 bp
Minimum required sequence length for both reads before a sequence pair gets removed: 10 bp
Running FastQC on the data once trimming has completed
Output file(s) will be GZIP compressed

Cutadapt seems to be fairly up-to-date (version 4.0). Setting -j 1
Writing f

## Align POINT-seq reads - STAR

### Create STAR index

In [8]:
!STAR --runMode genomeGenerate --genomeDir ./hg38_STAR_INDEX --genomeFastaFiles Homo_sapiens.GRCh38.dna.primary_assembly.fa --sjdbGTFfile Homo_sapiens.GRCh38.90.gtf --sjdbOverhang  149 --limitGenomeGenerateRAM 50000000000 --runThreadN 10

	STAR --runMode genomeGenerate --genomeDir ./hg38_STAR_INDEX --genomeFastaFiles Homo_sapiens.GRCh38.dna.primary_assembly.fa --sjdbGTFfile Homo_sapiens.GRCh38.90.gtf --sjdbOverhang 149 --limitGenomeGenerateRAM 50000000000 --runThreadN 10
	STAR version: 2.7.10a   compiled: 2022-01-14T18:50:00-05:00 :/home/dobin/data/STAR/STARcode/STAR.master/source
May 24 08:00:30 ..... started STAR run
May 24 08:00:30 ... starting to generate Genome files
May 24 08:01:41 ..... processing annotations GTF
May 24 08:02:18 ... starting to sort Suffix Array. This may take a long time...
May 24 08:02:34 ... sorting Suffix Array chunks and saving them to disk...
May 24 08:32:15 ... loading chunks from disk, packing SA...
May 24 08:38:34 ... finished generating suffix array
May 24 08:38:34 ... generating Suffix Array index
May 24 08:43:51 ... completed Suffix Array index
May 24 08:43:52 ..... inserting junctions into the genome indices
May 24 08:49:09 ... writing Genome to disk ...
May 24 08:49:37 ... writing S

### Align the reads

In [9]:
!STAR --genomeDir ./hg38_STAR_INDEX --outSAMtype BAM SortedByCoordinate --outFilterScoreMin 10 --readFilesCommand zcat  --readFilesIn SRR12802837_1_val_1.fq.gz SRR12802837_2_val_2.fq.gz --outFileNamePrefix SRR12802837 --outFilterMultimapNmax 1 --limitBAMsortRAM 50000000000 --runThreadN 20

	STAR --genomeDir ./hg38_STAR_INDEX --outSAMtype BAM SortedByCoordinate --outFilterScoreMin 10 --readFilesCommand zcat --readFilesIn SRR12802837_1_val_1.fq.gz SRR12802837_2_val_2.fq.gz --outFileNamePrefix SRR12802837 --outFilterMultimapNmax 1 --limitBAMsortRAM 50000000000 --runThreadN 20
	STAR version: 2.7.10a   compiled: 2022-01-14T18:50:00-05:00 :/home/dobin/data/STAR/STARcode/STAR.master/source
May 24 08:56:23 ..... started STAR run
May 24 08:56:23 ..... loading genome
May 24 08:57:58 ..... started mapping
May 24 09:14:43 ..... finished mapping
May 24 09:14:48 ..... started sorting BAM
May 24 09:23:37 ..... finished successfully


## Obtain reads' 5'end 

In [10]:
!python get_5SNR_POINT5.py SRR12802837Aligned.sortedByCoord.out.bam

Processing SRR12802837Aligned.sortedByCoord.out file!
[E::idx_find_and_load] Could not retrieve index file for 'SRR12802837Aligned.sortedByCoord.out.bam'
[E::idx_find_and_load] Could not retrieve index file for 'SRR12802837Aligned.sortedByCoord.out.bam'


## Filtering reads from the alignment file and split by biological strand

In [11]:
!samtools view -F 0x10 -o F_SRR12802837.bam 5pSNR_SRR12802837Aligned.sortedByCoord.out.bam
!samtools view -f 0x10 -o R_SRR12802837.bam 5pSNR_SRR12802837Aligned.sortedByCoord.out.bam

## Resume the genome-wide nascent signal

In [12]:
!bedtools genomecov -bga -split -ibam F_SRR12802837.bam | bedtools sort > F_SRR12802837.bg
!bedtools genomecov -bga -split -ibam R_SRR12802837.bam | bedtools sort > R_SRR12802837.bg

In [13]:
!samtools faidx Homo_sapiens.GRCh38.dna.primary_assembly.fa
!bedGraphToBigWig F_SRR12802837.bg Homo_sapiens.GRCh38.dna.primary_assembly.fa.fai F_SRR12802837.bw
!bedGraphToBigWig R_SRR12802837.bg Homo_sapiens.GRCh38.dna.primary_assembly.fa.fai R_SRR12802837.bw

In [14]:
!bedtools genomecov -bga -split -ibam F_SRR12802837.bam | bedtools sort > F_SRR12802837.bg
!bedtools genomecov -bga -split -ibam R_SRR12802837.bam | bedtools sort > R_SRR12802837.bg

In [15]:
!samtools faidx Homo_sapiens.GRCh38.dna.primary_assembly.fa
!bedGraphToBigWig F_SRR12802837.bg Homo_sapiens.GRCh38.dna.primary_assembly.fa.fai F_SRR12802837.bw
!bedGraphToBigWig R_SRR12802837.bg Homo_sapiens.GRCh38.dna.primary_assembly.fa.fai R_SRR12802837.bw